Imports the Hyperspy Function Library

In [1]:
import matplotlib
matplotlib.rcParams["backend"] = 'Qt4Agg'
import hyperspy.api as hs
import numpy as np
%matplotlib qt4

Opens a load window. Use hs.load('filename') if filename is known specifically.

In [30]:
wedge = hs.load()
wedge.set_signal_type('EELS')
wedge.metadata.General.title = 'Original Data Wedge'
wedge

<EELSSpectrum, title: Original Data Wedge, dimensions: (200, 5|2048)>

Check current metadata.

In [31]:
wedge.metadata

├── Acquisition_instrument
│   └── TEM
│       ├── Detector
│       │   └── EELS
│       │       └── collection_angle = 19.379844665527344
│       ├── beam_energy = 200.0
│       ├── convergence_angle = 25.0
│       └── dwell_time = 0.002
├── General
│   ├── original_filename = EELS Spectrum Image.dm3
│   └── title = Original Data Wedge
└── Signal
    ├── binned = True
    └── signal_type = EELS

In [4]:
wedge.plot()
wedge.axes_manager.gui()

Adjust the metadata for all the sample parameters.

In [32]:
wedge.metadata.Acquisition_instrument.TEM.dwell_time = 1
wedge.axes_manager[0].scale = 64
wedge.axes_manager[1].scale = 64

Extract t over lambda using estimate_thickness(point), where point is the first minimum point after the zero loss peak.

In [33]:
wedge.align_zero_loss_peak()

100%|██████████| 1000/1000 [00:00<00:00, 6306.71it/s]


Initial ZLP position statistics
-------------------------------
Summary statistics
------------------
mean:	-0.897
std:	2.489

min:	-5.200
Q1:	-3.000
median:	-0.900
Q3:	1.300
max:	3.400



100%|██████████| 1000/1000 [00:00<00:00, 6246.65it/s]


In [47]:
wedge.isig[:150].spikes_removal_tool()

In [27]:
s = wedge.estimate_elastic_scattering_threshold()

In [35]:

wedge.plot()

In [51]:
thickness = wedge.estimate_thickness(5.5)
thickness.plot()

# Convert from the intensity map to Line Profile
This assumes that the line profile is parallel to the x-axis in the image.

In [68]:
intensity = thickness
width = intensity.axes_manager[0].scale
line_scan = np.zeros(intensity.axes_manager.signal_shape[0])
line_scan.astype(float)
for row in range(intensity.axes_manager.signal_shape[1]):
    for col in range(intensity.axes_manager.signal_shape[0]):
        line_scan[col] = line_scan[col] + intensity.data[row, col]
    
line_scan = line_scan/intensity.axes_manager.signal_shape[1]
x_data = np.zeros(intensity.axes_manager.signal_shape[0])
x_data.astype(float)
for i in range(intensity.axes_manager.signal_shape[0]):
    x_data[i] = i*width

In [69]:
for i in range(len(line_scan)):
               print(line_scan[i])

0.0273050349206
0.071931026876
0.201623170823
0.39266422987
0.508926820755
0.536274582148
0.544236814976
0.553057360649
0.561177241802
0.567320179939
0.574176979065
0.588683390617
0.610193967819
0.622769320011
0.636157143116
0.634416556358
0.641336143017
0.606482183933
0.564242196083
0.562649035454
0.572112858295
0.590232872963
0.603809809685
0.606436097622
0.606679689884
0.615030515194
0.612078249454
0.609226918221
0.606913876534
0.598732352257
0.60426980257
0.609426629543
0.614152121544
0.61649056673
0.621168959141
0.628401255608
0.613984620571
0.612238705158
0.617731702328
0.632871246338
0.642433059216
0.65610922575
0.66052634716
0.668818485737
0.674740731716
0.679245400429
0.680319881439
0.676638913155
0.682396304607
0.692222356796
0.694784879684
0.704333031178
0.712172651291
0.71905374527
0.728275036812
0.735408782959
0.741739022732
0.74870647192
0.753234481812
0.756666207314
0.756549143791
0.76119145155
0.772667133808
0.777028143406
0.780767989159
0.777985930443
0.794904839993
0.

In [18]:
wedge.estimate_zero_loss_peak_centre()

<Signal2D, title: Original Data Wedge, dimensions: (|118, 5)>

In [29]:
wedge.plot()

In [11]:
wedge.save('Ni04EELS-processed')